In [2]:
import pymnet
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx


In [3]:
# Cell 2: Load Node and Edge Data from Excel Files
nodes_df = pd.read_excel('/Volumes/Data/NDSU/PhD Work/Research/IME Research/AI-Energy/OR Project/data/Nodes_subset.xlsx')
edges_df = pd.read_excel('/Volumes/Data/NDSU/PhD Work/Research/IME Research/AI-Energy/OR Project/data/Edges_subset.xlsx')


In [4]:
# Cell 3: Define Helper Functions and Major Categories
def is_major_category(row, voltage, operator):
    row_voltage = str(row['voltage']) if not pd.isna(row['voltage']) else ''
    row_operator = str(row['operator']) if not pd.isna(row['operator']) else ''
    return any(op in row_operator for op in operator.split(';')) and str(voltage) in row_voltage

# Define major operators and voltages for layering
major_operators = ['TenneT', '50Hertz']
major_voltages = [380000, 220000]


In [5]:
# Cell 4: Initialize MultiDiGraph and Add Nodes/Edges with Consistent Layer Attributes
import networkx as nx

G = nx.MultiDiGraph()

for operator in major_operators:
    for voltage in major_voltages:
        layer_name = f"{operator}_{voltage}"
        # Add nodes for this layer
        for index, node in nodes_df.iterrows():
            if is_major_category(node, voltage, operator):
                # Ensure all nodes have a 'layer' attribute
                node_attributes = node.to_dict()
                node_attributes['layer'] = layer_name
                G.add_node((node['v_id'], layer_name), **node_attributes)
        
        # Add edges for this layer
        for index, edge in edges_df.iterrows():
            if is_major_category(edge, voltage, operator):
                # Ensure both endpoints of the edge are already added as nodes
                if (edge['v_id_1'], layer_name) in G and (edge['v_id_2'], layer_name) in G:
                    # Ensure all edges have a 'layer' attribute
                    edge_attributes = edge.to_dict()
                    edge_attributes['layer'] = layer_name
                    G.add_edge((edge['v_id_1'], layer_name), (edge['v_id_2'], layer_name), **edge_attributes)


In [6]:
# Cell 5: Define and Add Inter-Layer Edges
def add_inter_layer_edges(G):
    nodes_per_layer = {}
    for node, data in G.nodes(data=True):
        node_id, layer = node
        if node_id not in nodes_per_layer:
            nodes_per_layer[node_id] = []
        nodes_per_layer[node_id].append(layer)
    
    for node_id, layers in nodes_per_layer.items():
        for i in range(len(layers)):
            for j in range(i + 1, len(layers)):
                G.add_edge((node_id, layers[i]), (node_id, layers[j]), inter_layer=True)

add_inter_layer_edges(G)


In [ ]:
# Cell 6: Visualization of the Network with Layer Differentiation and Safe Checks
def draw_network(G):
    plt.figure(figsize=(12, 8))
    pos = nx.spring_layout(G)  # Positioning the nodes based on Spring layout
    ax = plt.gca()

    # Define colors for each layer
    layer_colors = {
        'TenneT_380000': 'red',
        'TenneT_220000': 'blue',
        '50Hertz_380000': 'green',
        '50Hertz_220000': 'yellow'
    }
    
    # Draw nodes and edges with different colors for each layer
    for layer, color in layer_colors.items():
        # Extract nodes and edges for the current layer, checking for the 'layer' attribute
        layer_nodes = [(n, d) for n, d in G.nodes(data=True) if 'layer' in d and d['layer'] == layer]
        layer_edges = [(u, v, d) for u, v, d in G.edges(data=True) if 'layer' in d and d['layer'] == layer]
        
        # Node positions for current layer
        layer_pos = {node: pos[node] for node, data in layer_nodes}
        
        # Draw nodes and edges for current layer
        nx.draw_networkx_nodes(G, layer_pos, nodelist=[n for n, d in layer_nodes], 
                               node_color=color, node_size=100, ax=ax, label=f"{layer}")
        nx.draw_networkx_edges(G, layer_pos, edgelist=[(u, v) for u, v, d in layer_edges], 
                               width=2, edge_color=color)
    
    # Display legend
    plt.legend(title="Layer", title_fontsize='13', fontsize='12')
    
    plt.title("Visual Representation of the Multiplex Network")
    plt.show()

draw_network(G)


In [8]:
import pulp

# Extract nodes and layers from the graph
nodes = {node for node, data in G.nodes(data=True)}
layers = {data['layer'] for node, data in G.nodes(data=True)}

# Assuming a set number of clusters (define K as per your requirement)
K = 4  # Number of clusters you want to consider

# Initialize the optimization model
model = pulp.LpProblem("Multiplex_Network_Clustering", pulp.LpMinimize)

# Define decision variables for node-cluster assignments and diameters
x = pulp.LpVariable.dicts("node_cluster",
                          ((node, cluster_id, layer) for node in nodes
                           for cluster_id in range(K) for layer in layers),
                          cat=pulp.LpBinary)

d = pulp.LpVariable.dicts("diameter",
                          ((cluster_id, layer) for cluster_id in range(K) for layer in layers),
                          lowBound=0, cat=pulp.LpContinuous)

D_max = pulp.LpVariable("D_max", lowBound=0, cat=pulp.LpContinuous)


In [9]:
# Calculate shortest path distances for each layer
d_ij = {}  # Dictionary to store shortest path distances

for layer in layers:
    subgraph = nx.subgraph_view(G, filter_node=lambda n: G.nodes[n]['layer'] == layer)
    # Calculate shortest paths for all pairs in this subgraph
    path_lengths = dict(nx.all_pairs_dijkstra_path_length(subgraph))
    for node1 in subgraph.nodes():
        for node2 in subgraph.nodes():
            # Store the shortest path length in d_ij, ensuring node1 and node2 are ordered
            if node1 != node2:
                d_ij[(node1, node2, layer)] = path_lengths[node1].get(node2, float('inf'))

# Now d_ij[(node1, node2, layer)] holds the shortest path distance between node1 and node2 within the same layer
                for layer in layers:
    subgraph = nx.subgraph_view(G, filter_node=lambda n: G.nodes[n]['layer'] == layer)
    # Calculate shortest paths for all pairs in this subgraph
    path_lengths = dict(nx.all_pairs_dijkstra_path_length(subgraph))
    for node1 in subgraph.nodes():
        for node2 in subgraph.nodes():
            if node1 != node2:
                # Ensure every possible pair is accounted for, even if not directly connected
                d_ij[(node1, node2, layer)] = path_lengths[node1].get(node2, float('inf'))
            else:
                # Include distance to self as 0 to ensure key exists
                d_ij[(node1, node2, layer)] = 0



In [10]:
# Membership constraints: every node must be assigned to exactly one cluster per layer
for node in nodes:
    for layer in layers:
        model += pulp.lpSum(x[node, cluster_id, layer] for cluster_id in range(K)) == 1

# Diameter constraints
for layer in layers:
    for cluster_id in range(K):
        for node1 in nodes:
            for node2 in nodes:
                if node1 != node2:
                    # Ensure that the diameter d is at least as large as the distance between any two nodes in the same cluster
                    model += d[cluster_id, layer] >= d_ij[(node1, node2, layer)] * (x[node1, cluster_id, layer] + x[node2, cluster_id, layer] - 1)

# Define D_max as the maximum diameter across all clusters and layers
for layer in layers:
    for cluster_id in range(K):
        model += D_max >= d[cluster_id, layer]

# Set the objective
model += D_max

# Solve the problem
model.solve()


KeyError: ((22, '50Hertz_220000'), (38, '50Hertz_380000'), 'TenneT_220000')

In [13]:
missing_keys = []
for layer in layers:
    for node1 in nodes:
        for node2 in nodes:
            if node1 != node2:  # only considering pairs of different nodes
                key = (node1, node2, layer)
                if key not in d_ij:
                    missing_keys.append(key)

print("Missing keys:", missing_keys)


Missing keys: [((22, '50Hertz_220000'), (38, '50Hertz_380000'), 'TenneT_220000'), ((22, '50Hertz_220000'), (4, 'TenneT_380000'), 'TenneT_220000'), ((22, '50Hertz_220000'), (13, 'TenneT_220000'), 'TenneT_220000'), ((22, '50Hertz_220000'), (2, 'TenneT_220000'), 'TenneT_220000'), ((22, '50Hertz_220000'), (24, 'TenneT_220000'), 'TenneT_220000'), ((22, '50Hertz_220000'), (35, 'TenneT_220000'), 'TenneT_220000'), ((22, '50Hertz_220000'), (18, '50Hertz_380000'), 'TenneT_220000'), ((22, '50Hertz_220000'), (15, 'TenneT_220000'), 'TenneT_220000'), ((22, '50Hertz_220000'), (26, 'TenneT_220000'), 'TenneT_220000'), ((22, '50Hertz_220000'), (20, '50Hertz_380000'), 'TenneT_220000'), ((22, '50Hertz_220000'), (40, '50Hertz_380000'), 'TenneT_220000'), ((22, '50Hertz_220000'), (11, '50Hertz_220000'), 'TenneT_220000'), ((22, '50Hertz_220000'), (6, 'TenneT_220000'), 'TenneT_220000'), ((22, '50Hertz_220000'), (44, '50Hertz_220000'), 'TenneT_220000'), ((22, '50Hertz_220000'), (22, '50Hertz_380000'), 'TenneT_2